In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import sys
import re

sys.path.append("../")
import dataset_handler as dh
import helper_func as hf

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [15]:
name = "gsm8k_fewer_alt_codegen_func"

file = "data/priming_texts/gsm8k/"+ name +".txt"


with open(file, "r") as f:
    prompt = f.read()
#gptj_model = "EleutherAI/gpt-j-6B"

In [3]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
#model_args.model = "codegen-350M-mono"
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 435.05s
loading tokenizer
loading tokenizer took 4.07s


In [ ]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [13]:
def preproc_gen_toks(gen_toks, input_len, tokenizer):
    """Process generated tokens from model keeping only up to \n\n

    :param list of list gen_toks: the output form the model
    :param int input_len: input lenght used to ignore the prompt
    :param HF_tokenizer tokenizer: tokenizer used for decoding
    :return list of str : list of generated outputs
    """
    list_out = []
    for gen_tok in gen_toks:
        last_tokens = gen_tok[input_len:]
        generated_text = tokenizer.decode(last_tokens)
        #print(generated_text)
        print_pattern = re.compile(r"float\(([^)]+)\)")
        split_list = re.split(print_pattern, generated_text)
        if len(split_list) > 1:
            output = f"{split_list[0]}float({split_list[1]})\n"
        else:
            output = "INVALID OUTPUT"
        # output = generated_text.split("\n\n")[0]
        list_out.append(output)
    return list_out

In [6]:
config = hf.codegen_gen_args()
config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.k = 3
config.max_lenght_after_input = 250
config.top_p = 0.95
config.top_k = 50
config.temperature = 0.7
config.min_length = 3

gen_args = config

In [21]:
tokens = tokenizer(prompt, return_tensors="pt").input_ids
generated_tokens = model.generate(
                tokens.long().cuda(),
                use_cache=True,
                do_sample=True,
                top_k=gen_args.top_k,
                temperature=gen_args.temperature,
                top_p=gen_args.top_p,
                min_length=len(tokens[0]) + gen_args.min_length,
                max_length=len(tokens[0]) + gen_args.max_length_after_input,
                num_return_sequences=gen_args.num_return_sequences,
                pad_token_id=tokenizer.eos_token_id,
            )

list_outputs = preproc_gen_toks(generated_tokens, len(tokens[0]), tokenizer)
print(list_outputs[0])

RuntimeError: CUDA out of memory. Tried to allocate 168.00 MiB (GPU 0; 47.54 GiB total capacity; 43.55 GiB already allocated; 50.75 MiB free; 45.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
double_tokens = tokenizer([prompt, prompt], return_tensors="pt").input_ids

In [12]:
print(list_outputs[2])
exec(list_outputs[1])

 3+5=8
3*5=15
5*15=75
print(3+5)


IndentationError: unexpected indent (<string>, line 1)